# Quantum Ising Model
Following a notebook by [Carsten Bauer & Katharine Hyatt](https://github.com/crstnbr/JuliaOulu20/blob/master/Day2/4_linear_algebra/2_ed_quantum_ising.ipynb).

The [Ising Model](https://en.wikipedia.org/wiki/Ising_model) is a model of ferromagnetism, which considers localized discrete spins of a quantum system.

We will study the Ising model in terms of the time-independent Schrödinger equation
$$
H \left|{ \psi }\right\rangle = E \left|{ \psi }\right\rangle,
$$

where $H$ is to be diagonalized by means of a computational model.

## Transverse field Ising chain
We consider the general Ising hamiltonian function

$$
\mathscr{H} = - \sum_{\left\langle i,j \right\rangle} J_{ij}\sigma_i \sigma_j - \mu \sum_j h_j \sigma_j
$$

for any two adjacent crystal sites $i, j$, with interaction $J_{i,j}$ between sites, and external magnetic field $h_j$. The first sum is across pairs of adjecent spins, with notation $\left\langle i,j \right\rangle$ denoting nearest neighbour.

We will denote the spin states through the Pauli matrices, with the external magnetic field acting perpendicular to the spin orientations -- i.e., we use $\sigma^z$ for spin directions (the first summation term), and $\sigma^x$ for interaction with the magnetic field (second summation term)

For our notation, we will combine $\mu h_j =: h$, $\forall j$.

We define

In [1]:
σᶻ = [1 0; 0 -1]
σˣ = [0 1; 1 0]

σᶻ, σˣ

([1 0; 0 -1], [0 1; 1 0])

We denote the Eigenvectors of $\sigma^z$ by $\left|{ \uparrow }\right\rangle$, $\left|{ \downarrow }\right\rangle$ respective to their orientation along the $z$ axis -- these will also denote our spin orientations.

We can then see that the function of $\sigma^x$ is to flip the orientation, as it is purely off-diagonal:

In [2]:
using LinearAlgebra

evecs = eigen(σᶻ).vectors
s_down = evecs[1, :]
s_up = evecs[2, :]

@show s_up, s_down

s_up == σˣ * s_down

(s_up, s_down) = ([1.0, 0.0], [0.0, 1.0])


true

Expressed symbolically:
$$
\sigma^x \left|{ \uparrow }\right\rangle = \left|{ \downarrow }\right\rangle.
$$

## Hamiltonian construction

In the case of a trivial (diagonal) Hamiltonian (i.e. no external magnetic field $h=0$), we have a classical system where the spins are flipped and disordered by thermal fluctuations.

Such a system exhibits different phases at different temperatures $T$ relative to a critical transition temperature $T_c$:
- the *paramagnetic* phase ($T > T_c$)
- the *ferromagnetic* phase ($T < T_c$)

with an implicit ferromagnetic ground state $T=0$.


By introducing $h\neq0$, we construct a Hamiltonian matrix with off-diagonal elements contributed from $\sigma^x$ terms. These elements represent the quantum fluctuations of the system, by changing some spins orientated along $z$ to $x$.


For now, let us choose $\sigma^z$ as our Eigenbasis for computation.

We will use the Kronecker product

In [3]:
kron(σᶻ,σᶻ)

4×4 Matrix{Int64}:
 1   0   0  0
 0  -1   0  0
 0   0  -1  0
 0   0   0  1

which we can abreviate with 

In [11]:
⊗(x, y) = kron(x, y)
# identities
⊗(x, y::UniformScaling) = x ⊗ [1 0; 0 1]
⊗(x::UniformScaling, y) = [1 0; 0 1] ⊗ y
⊗(x::UniformScaling, y::UniformScaling) = [1 0; 0 1] ⊗ [1 0; 0 1]

⊗ (generic function with 4 methods)

In [12]:
σᶻ ⊗ σᶻ

4×4 Matrix{Int64}:
 1   0   0  0
 0  -1   0  0
 0   0  -1  0
 0   0   0  1

The next step is to calculate permutations of the summation terms, so we can assemble our Hamiltonian as a $2^n \times 2^n$ matrix, representing a quantum chain of $n$ particles:

In [23]:
function isingfield(;N::Int, h)
    σᶻ = [1 0; 0 -1]
    σˣ = [0 1; 1 0]
    
    # assemble terms
    terms = vcat([σᶻ, σᶻ], fill(I, N-2))
    
    H = zeros(Int, 2^N, 2^N)
    
    # compute first sum
    for i in 1:N-1
        H -= foldl(⊗, terms)
        terms = circshift(terms, 1)
    end
    
    # sum two terms
    terms[1] = σˣ
    terms[2] = I
    
    # compute second sum
    
    for i in 1:N
        H -= h * foldl(⊗, terms)
        terms = circshift(terms, 1)
    end
    
    H
end

isingfield(N=2, h=0.5)

4×4 Matrix{Float64}:
 -1.0  -0.5  -0.5   0.0
 -0.5   1.0   0.0  -0.5
 -0.5   0.0   1.0  -0.5
  0.0  -0.5  -0.5  -1.0

## Basis states

We will choose 0 (false) to represent down spins, and 1 (true) to represent up spins. We can then write basis states in the notation
$$
\left\lvert 0010 \right\rangle = \left\lvert \downarrow  \downarrow \uparrow \downarrow \right\rangle
$$
for e.g. a 4 site system.

The full basis is then just a counting problem:

In [34]:
# helper function
binrep(i::Int, pad::Int) = BitArray(parse(Bool, j) for j in string(i, base=2, pad=pad))

function makebasis(N::Int)
    nstates = 2^N
    basis = Vector{BitArray{1}}(undef, nstates)
    
    for i in 0:nstates-1
        basis[i+1] = binrep(i, N)
    end
    
    basis
end     

makebasis(4)

16-element Vector{BitVector}:
 [0, 0, 0, 0]
 [0, 0, 0, 1]
 [0, 0, 1, 0]
 [0, 0, 1, 1]
 [0, 1, 0, 0]
 [0, 1, 0, 1]
 [0, 1, 1, 0]
 [0, 1, 1, 1]
 [1, 0, 0, 0]
 [1, 0, 0, 1]
 [1, 0, 1, 0]
 [1, 0, 1, 1]
 [1, 1, 0, 0]
 [1, 1, 0, 1]
 [1, 1, 1, 0]
 [1, 1, 1, 1]

## Diagonalization
We solve the Schrödinger equation by diagonalizing $\mathscr{H}$. We will use $N=3$, and $h=1$:

In [134]:
basis = makebasis(3)
H = isingfield(N=3, h=1)

vals, vecs = eigen(H)
vals

8-element Vector{Float64}:
 -2.9093129111764067
 -2.9093129111763933
 -1.2393136749274754
 -1.2393136749274718
  1.2393136749274782
  1.239313674927481
  2.909312911176408
  2.9093129111764093

We can read off the ground state as a probability amplitude for each basis vector

In [135]:
groundstate = vecs[:, 1]

8-element Vector{Float64}:
  0.0
 -0.05673591158528354
  0.0
 -0.3200679131671591
  0.056735911585283544
 -0.15500539306473066
 -0.3200679131671591
  0.8744418006452241

We can pretty print this with some not so pretty string formatting:

In [136]:
using LaTeXStrings

# fmt function for kets
function fmt(x::BitVector)
    s = join(i == false ? "\\downarrow" : "\\uparrow" for i in x)
    "\\left\\lvert $s \\right\\rangle"
end

# fmt function for results
function fmt(basis::Vector{BitVector}, groundstate::Vector)
    out = String["\\text{Spin State: Probability}"]
    
    for (b, p) in zip(basis, groundstate)
        push!(out, "$(fmt(b)): $(abs2(p))")
    end
    
    latexstring(join(out, "\\\\"))
end

fmt(basis, groundstate)

L"$\text{Spin State: Probability}\\\left\lvert \downarrow\downarrow\downarrow \right\rangle: 0.0\\\left\lvert \downarrow\downarrow\uparrow \right\rangle: 0.0032189636634131105\\\left\lvert \downarrow\uparrow\downarrow \right\rangle: 0.0\\\left\lvert \downarrow\uparrow\uparrow \right\rangle: 0.1024434690391801\\\left\lvert \uparrow\downarrow\downarrow \right\rangle: 0.0032189636634131114\\\left\lvert \uparrow\downarrow\uparrow \right\rangle: 0.02402667187915165\\\left\lvert \uparrow\uparrow\downarrow \right\rangle: 0.1024434690391801\\\left\lvert \uparrow\uparrow\uparrow \right\rangle: 0.7646484627156619$"

We can try this again with $h=0$ to see the case of a trivial Hamiltonian:

In [137]:
basis = makebasis(3)
H = isingfield(N=3, h=0)

vals, vecs = eigen(H)
groundstate = vecs[:, 1]

fmt(basis, groundstate)

L"$\text{Spin State: Probability}\\\left\lvert \downarrow\downarrow\downarrow \right\rangle: 1.0\\\left\lvert \downarrow\downarrow\uparrow \right\rangle: 0.0\\\left\lvert \downarrow\uparrow\downarrow \right\rangle: 0.0\\\left\lvert \downarrow\uparrow\uparrow \right\rangle: 0.0\\\left\lvert \uparrow\downarrow\downarrow \right\rangle: 0.0\\\left\lvert \uparrow\downarrow\uparrow \right\rangle: 0.0\\\left\lvert \uparrow\uparrow\downarrow \right\rangle: 0.0\\\left\lvert \uparrow\uparrow\uparrow \right\rangle: 0.0$"

As we expect, all of the spin states are aligned along the $-z$ axis.